In [0]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [0]:
!pip install --upgrade -q gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [5]:
# Open our new sheet and read some data.
worksheet = gc.open('web_scarping_csv_140').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df=pd.DataFrame.from_records(rows)
df.head(n=4)

[['', 'header', 'sub_header', 'text_content', 'author'], ['0', 'Bowies konstskolahar stängt för gott', '', '\nDaniel Swedin: Rockens kameleont gör sorti med en fantastisk slutscen\nSå blev hans senaste skiva hans sista.\nFör en artist så upptagen med spelet mellan liv och död, uppgång och fall, så borde vi kanske ta emot budet om David Bowies bortgång med ett "förstås!"\nDet är en fantastisk slutscen i den popteater han i 45 år gjort till sin karriär. Singeln från skivan heter ju till och med Lazarus, en man som enligt Johannesevangeliet blir uppväckt från de döda av Jesus.\nMen det finns faktiskt ingenting som kunde förbereda mig på det här. Han är borta. Det blir inget mer.\n"En enmanskonstskola för ungar som aldrig kom in på någon konstskola". Så beskrev filmkritikern Stephen Dalton den brittiske popartisten i en text i julas.\nDet är ingen dum formulering. Det var via Bowie jag hittade vidare till Hanif Kureishi, O\'Jays, Fritz Lang, Collin MacInnes, Scott Walker, Kenneth Anger, Je

,0,1,2,3,4
0,,header,sub_header,text_content,author
1,0,Bowies konstskolahar stängt för gott,,\nDaniel Swedin: Rockens kameleont gör sorti m...,DANIEL SWEDIN
2,1,Matfusket är en miljardaffär,,\nDitt kravmärkta svenska kött kan i själva ve...,DANIEL SWEDIN
3,2,Lågavlönade ska packa Ica-kassen,,\nDe enkla jobben är för få och lönerna är för...,DANIEL SWEDIN


In [6]:
df.columns = df.iloc[0]
df=df.drop([0])
df.head(n=5)

,,header,sub_header,text_content,author
1,0,Bowies konstskolahar stängt för gott,,\nDaniel Swedin: Rockens kameleont gör sorti m...,DANIEL SWEDIN
2,1,Matfusket är en miljardaffär,,\nDitt kravmärkta svenska kött kan i själva ve...,DANIEL SWEDIN
3,2,Lågavlönade ska packa Ica-kassen,,\nDe enkla jobben är för få och lönerna är för...,DANIEL SWEDIN
4,3,Samhället måste våga möta hatet,,\nEfter bombdådet i Oslo och massakern på Utøy...,DANIEL SWEDIN
5,4,Vi måste knäcka IS på hemmaplan,,\n■ Efter terrorattentaten i Bryssel är det en...,DANIEL SWEDIN


In [0]:
df['text_content'] = df['text_content'].str.replace("[^a-zåäöA-ZÅÄÖ]", " ")


In [8]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('swedish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# tokenization 
df['text_content'] = [word for word in df['text_content'].tolist() if word.lower() not in stop_words]


In [0]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
trn_df,val_df = train_test_split(df, test_size=0.2)


trn_texts = trn_df['text_content']
val_texts = val_df['text_content']

trn_labels = trn_df['author']
val_labels = val_df['author']


col_names = ['labels','text']


df_trn = pd.DataFrame({'text':trn_texts, 'labels':trn_labels}, columns=col_names)
df_val = pd.DataFrame({'text':val_texts, 'labels':val_labels}, columns=col_names)


In [0]:
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")


In [0]:
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=8)

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.7)

In [28]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,8.592543,6.077806,0.140402,01:48


In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM,drop_mult=0.7)
learn.load_encoder('ft_enc')

In [36]:
learn.fit_one_cycle(1, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,1.177868,1.127667,0.285714,03:06
